In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.decomposition import PCA



from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
accepted = pd.read_csv('accepted_2007_to_2018_100T.csv' )

C:\Users\hannan\anaconda3\envs\mlProject\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
rejected = pd.read_csv('rejected_2007_to_2018_100T.csv', low_memory=False, encoding="ISO-8859-1")

In [4]:
rejected

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code;;
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0;;
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0;;
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0;;
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0;;
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0;;
...,...,...,...,...,...,...,...,...,...
99996,25000.0,2010-03-10,small_business,670.0,26.96%,021xx,MA,10+ years,0.0;;
99997,4000.0,2010-03-10,credit_card,676.0,64.48%,184xx,PA,10+ years,0.0;;
99998,5000.0,2010-03-10,debt_consolidation,606.0,21.9%,851xx,AZ,3 years,0.0;;
99999,2800.0,2010-03-10,debt_consolidation,727.0,31.33%,339xx,FL,5 years,0.0;;


In [5]:
rejected = rejected.dropna(axis=0).reset_index(drop=True)

In [6]:
rejected.isna().sum().sum()

0

In [7]:

rejected['Risk_Score'] = rejected['Risk_Score'].astype(float)

In [8]:
rejected['Amount Requested'] = rejected['Amount Requested'].astype(float)

In [9]:
rejected['Debt-To-Income Ratio']= rejected['Debt-To-Income Ratio'].apply(lambda x: x[0:-1])

In [10]:
rejected['Debt-To-Income Ratio'] = rejected['Debt-To-Income Ratio'].astype(float)

In [11]:
rejected.loc[4, 'Policy Code;;'][0:1]

'0'

In [12]:
rejected['Policy Code;;']= rejected['Policy Code;;'].apply(lambda x: x[0:1])

In [13]:
rejected

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code;;
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10.00,481xx,NM,4 years,0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10.00,010xx,MA,< 1 year,0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10.00,212xx,MD,1 year,0
3,6000.0,2007-05-27,waksman,698.0,38.64,017xx,MA,< 1 year,0
4,1500.0,2007-05-27,mdrigo,509.0,9.43,209xx,MD,< 1 year,0
...,...,...,...,...,...,...,...,...,...
88940,25000.0,2010-03-10,small_business,670.0,26.96,021xx,MA,10+ years,0
88941,4000.0,2010-03-10,credit_card,676.0,64.48,184xx,PA,10+ years,0
88942,5000.0,2010-03-10,debt_consolidation,606.0,21.90,851xx,AZ,3 years,0
88943,2800.0,2010-03-10,debt_consolidation,727.0,31.33,339xx,FL,5 years,0


In [14]:
rejected['Policy Code;;'] = rejected['Policy Code;;'].astype(float)

In [15]:
rejected.dtypes

Amount Requested        float64
Application Date         object
Loan Title               object
Risk_Score              float64
Debt-To-Income Ratio    float64
Zip Code                 object
State                    object
Employment Length        object
Policy Code;;           float64
dtype: object

In [16]:
rejected = rejected.rename(columns={"Debt-To-Income Ratio": "dti", "Amount Requested": "loan_amnt","State": "addr_state"})

In [17]:
rejected = rejected.rename(columns={"Employment Length": "emp_length", "Policy Code;;": "policy_code"})

In [18]:
rejected


,loan_amnt,Application Date,Loan Title,Risk_Score,dti,Zip Code,addr_state,emp_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10.00,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10.00,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10.00,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43,209xx,MD,< 1 year,0.0
...,...,...,...,...,...,...,...,...,...
88940,25000.0,2010-03-10,small_business,670.0,26.96,021xx,MA,10+ years,0.0
88941,4000.0,2010-03-10,credit_card,676.0,64.48,184xx,PA,10+ years,0.0
88942,5000.0,2010-03-10,debt_consolidation,606.0,21.90,851xx,AZ,3 years,0.0
88943,2800.0,2010-03-10,debt_consolidation,727.0,31.33,339xx,FL,5 years,0.0


In [19]:
accepted = accepted[['loan_amnt','fico_range_low', 'fico_range_high', 'dti', 'addr_state', 'emp_length', 'loan_status',  
                     ]]
accepted.dropna().head(10)

,loan_amnt,fico_range_low,fico_range_high,dti,addr_state,emp_length,loan_status
0,3600.0,675.0,679.0,5.91,PA,10+ years,Fully Paid
1,24700.0,715.0,719.0,16.06,SD,10+ years,Fully Paid
2,20000.0,695.0,699.0,10.78,IL,10+ years,Fully Paid
3,35000.0,785.0,789.0,17.06,NJ,10+ years,Current
4,10400.0,695.0,699.0,25.37,PA,3 years,Fully Paid
5,11950.0,690.0,694.0,10.20,GA,4 years,Fully Paid
6,20000.0,680.0,684.0,14.67,MN,10+ years,Fully Paid
7,20000.0,705.0,709.0,17.61,SC,10+ years,Fully Paid
8,10000.0,685.0,689.0,13.07,PA,6 years,Fully Paid
9,8000.0,700.0,704.0,34.80,RI,10+ years,Fully Paid


In [20]:
accepted['loan_status'].unique()

array(['Fully Paid', 'Current', 'Charged Off', nan, 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default'], dtype=object)

In [21]:
accepted

,loan_amnt,fico_range_low,fico_range_high,dti,addr_state,emp_length,loan_status
0,3600.0,675.0,679.0,5.91,PA,10+ years,Fully Paid
1,24700.0,715.0,719.0,16.06,SD,10+ years,Fully Paid
2,20000.0,695.0,699.0,10.78,IL,10+ years,Fully Paid
3,35000.0,785.0,789.0,17.06,NJ,10+ years,Current
4,10400.0,695.0,699.0,25.37,PA,3 years,Fully Paid
...,...,...,...,...,...,...,...
99997,14000.0,665.0,669.0,23.43,NC,3 years,Fully Paid
99998,30000.0,730.0,734.0,27.92,PA,NaN,Fully Paid
99999,24000.0,705.0,709.0,19.26,IN,3 years,Current
100000,6000.0,685.0,689.0,20.83,NE,4 years,Charged Off


In [22]:
accepted = accepted[accepted.loan_status != 'Current']
accepted = accepted[accepted.loan_status != 'Late (31-120 days)']
accepted = accepted[accepted.loan_status != 'In Grace Period']
accepted = accepted[accepted.loan_status != 'Late (16-30 days)']

In [23]:
accepted

,loan_amnt,fico_range_low,fico_range_high,dti,addr_state,emp_length,loan_status
0,3600.0,675.0,679.0,5.91,PA,10+ years,Fully Paid
1,24700.0,715.0,719.0,16.06,SD,10+ years,Fully Paid
2,20000.0,695.0,699.0,10.78,IL,10+ years,Fully Paid
4,10400.0,695.0,699.0,25.37,PA,3 years,Fully Paid
5,11950.0,690.0,694.0,10.20,GA,4 years,Fully Paid
...,...,...,...,...,...,...,...
99996,3000.0,695.0,699.0,19.23,FL,8 years,Fully Paid
99997,14000.0,665.0,669.0,23.43,NC,3 years,Fully Paid
99998,30000.0,730.0,734.0,27.92,PA,NaN,Fully Paid
100000,6000.0,685.0,689.0,20.83,NE,4 years,Charged Off


In [24]:
accepted['Risk_Score'] = accepted.loc[:, ['fico_range_low','fico_range_high']].mean(axis=1)

In [25]:
accepted

,loan_amnt,fico_range_low,fico_range_high,dti,addr_state,emp_length,loan_status,Risk_Score
0,3600.0,675.0,679.0,5.91,PA,10+ years,Fully Paid,677.0
1,24700.0,715.0,719.0,16.06,SD,10+ years,Fully Paid,717.0
2,20000.0,695.0,699.0,10.78,IL,10+ years,Fully Paid,697.0
4,10400.0,695.0,699.0,25.37,PA,3 years,Fully Paid,697.0
5,11950.0,690.0,694.0,10.20,GA,4 years,Fully Paid,692.0
...,...,...,...,...,...,...,...,...
99996,3000.0,695.0,699.0,19.23,FL,8 years,Fully Paid,697.0
99997,14000.0,665.0,669.0,23.43,NC,3 years,Fully Paid,667.0
99998,30000.0,730.0,734.0,27.92,PA,NaN,Fully Paid,732.0
100000,6000.0,685.0,689.0,20.83,NE,4 years,Charged Off,687.0


In [26]:
accepted = accepted.rename(columns={"loan_status": "policy_code"})

In [27]:
accepted

,loan_amnt,fico_range_low,fico_range_high,dti,addr_state,emp_length,policy_code,Risk_Score
0,3600.0,675.0,679.0,5.91,PA,10+ years,Fully Paid,677.0
1,24700.0,715.0,719.0,16.06,SD,10+ years,Fully Paid,717.0
2,20000.0,695.0,699.0,10.78,IL,10+ years,Fully Paid,697.0
4,10400.0,695.0,699.0,25.37,PA,3 years,Fully Paid,697.0
5,11950.0,690.0,694.0,10.20,GA,4 years,Fully Paid,692.0
...,...,...,...,...,...,...,...,...
99996,3000.0,695.0,699.0,19.23,FL,8 years,Fully Paid,697.0
99997,14000.0,665.0,669.0,23.43,NC,3 years,Fully Paid,667.0
99998,30000.0,730.0,734.0,27.92,PA,NaN,Fully Paid,732.0
100000,6000.0,685.0,689.0,20.83,NE,4 years,Charged Off,687.0


In [28]:
accepted = accepted[['loan_amnt','Risk_Score', 'dti', 'addr_state', 'emp_length', 'policy_code',  
                     ]]
accepted.dropna().head(10)

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,3600.0,677.0,5.91,PA,10+ years,Fully Paid
1,24700.0,717.0,16.06,SD,10+ years,Fully Paid
2,20000.0,697.0,10.78,IL,10+ years,Fully Paid
4,10400.0,697.0,25.37,PA,3 years,Fully Paid
5,11950.0,692.0,10.20,GA,4 years,Fully Paid
6,20000.0,682.0,14.67,MN,10+ years,Fully Paid
7,20000.0,707.0,17.61,SC,10+ years,Fully Paid
8,10000.0,687.0,13.07,PA,6 years,Fully Paid
9,8000.0,702.0,34.80,RI,10+ years,Fully Paid
12,1400.0,702.0,34.95,NC,3 years,Fully Paid


In [29]:
accepted['policy_code'].unique()

array(['Fully Paid', 'Charged Off', nan, 'Default'], dtype=object)

In [30]:
accepted = accepted[accepted.policy_code != 'Default']

In [31]:
accepted = accepted.dropna(axis=0).reset_index(drop=True)

In [32]:
accepted['policy_code'].unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

In [33]:
accepted

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,3600.0,677.0,5.91,PA,10+ years,Fully Paid
1,24700.0,717.0,16.06,SD,10+ years,Fully Paid
2,20000.0,697.0,10.78,IL,10+ years,Fully Paid
3,10400.0,697.0,25.37,PA,3 years,Fully Paid
4,11950.0,692.0,10.20,GA,4 years,Fully Paid
...,...,...,...,...,...,...
81436,20000.0,707.0,25.77,MO,8 years,Fully Paid
81437,3000.0,697.0,19.23,FL,8 years,Fully Paid
81438,14000.0,667.0,23.43,NC,3 years,Fully Paid
81439,6000.0,687.0,20.83,NE,4 years,Charged Off


In [34]:

def binary_encode(df, column, positive_value):
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    
    


    


In [35]:
binary_encode(accepted, 'policy_code', 'Fully Paid')

In [36]:
accepted

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,3600.0,677.0,5.91,PA,10+ years,1
1,24700.0,717.0,16.06,SD,10+ years,1
2,20000.0,697.0,10.78,IL,10+ years,1
3,10400.0,697.0,25.37,PA,3 years,1
4,11950.0,692.0,10.20,GA,4 years,1
...,...,...,...,...,...,...
81436,20000.0,707.0,25.77,MO,8 years,1
81437,3000.0,697.0,19.23,FL,8 years,1
81438,14000.0,667.0,23.43,NC,3 years,1
81439,6000.0,687.0,20.83,NE,4 years,0


In [37]:
accepted['policy_code'].unique()

array([1, 0], dtype=int64)

In [38]:
rejected

,loan_amnt,Application Date,Loan Title,Risk_Score,dti,Zip Code,addr_state,emp_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10.00,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10.00,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10.00,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43,209xx,MD,< 1 year,0.0
...,...,...,...,...,...,...,...,...,...
88940,25000.0,2010-03-10,small_business,670.0,26.96,021xx,MA,10+ years,0.0
88941,4000.0,2010-03-10,credit_card,676.0,64.48,184xx,PA,10+ years,0.0
88942,5000.0,2010-03-10,debt_consolidation,606.0,21.90,851xx,AZ,3 years,0.0
88943,2800.0,2010-03-10,debt_consolidation,727.0,31.33,339xx,FL,5 years,0.0


In [39]:
rejected = rejected[['loan_amnt','Risk_Score', 'dti', 'addr_state', 'emp_length', 'policy_code',  
                     ]]
rejected.dropna().head(10)

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,1000.0,693.0,10.00,NM,4 years,0.0
1,1000.0,703.0,10.00,MA,< 1 year,0.0
2,11000.0,715.0,10.00,MD,1 year,0.0
3,6000.0,698.0,38.64,MA,< 1 year,0.0
4,1500.0,509.0,9.43,MD,< 1 year,0.0
5,15000.0,645.0,0.00,NY,3 years,0.0
6,10000.0,693.0,10.00,MD,< 1 year,0.0
7,3900.0,700.0,10.00,IN,2 years,0.0
8,3000.0,694.0,10.00,CO,4 years,0.0
9,2500.0,573.0,11.76,KY,4 years,0.0


In [40]:
binary_encode(rejected, 'policy_code', '1')

<ipython-input-34-094600b3610b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)


In [41]:
rejected

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,1000.0,693.0,10.00,NM,4 years,0
1,1000.0,703.0,10.00,MA,< 1 year,0
2,11000.0,715.0,10.00,MD,1 year,0
3,6000.0,698.0,38.64,MA,< 1 year,0
4,1500.0,509.0,9.43,MD,< 1 year,0
...,...,...,...,...,...,...
88940,25000.0,670.0,26.96,MA,10+ years,0
88941,4000.0,676.0,64.48,PA,10+ years,0
88942,5000.0,606.0,21.90,AZ,3 years,0
88943,2800.0,727.0,31.33,FL,5 years,0


In [42]:
frames= [accepted,rejected]

In [43]:
combined = pd.concat(frames)

In [44]:
combined

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,3600.0,677.0,5.91,PA,10+ years,1
1,24700.0,717.0,16.06,SD,10+ years,1
2,20000.0,697.0,10.78,IL,10+ years,1
3,10400.0,697.0,25.37,PA,3 years,1
4,11950.0,692.0,10.20,GA,4 years,1
...,...,...,...,...,...,...
88940,25000.0,670.0,26.96,MA,10+ years,0
88941,4000.0,676.0,64.48,PA,10+ years,0
88942,5000.0,606.0,21.90,AZ,3 years,0
88943,2800.0,727.0,31.33,FL,5 years,0


In [45]:
combined.isna().sum().sum()

0

In [46]:

def onehot_encode(df, columns):

    df = df.copy()
    for column in columns:
        dummies = pd.get_dummies(df[column], prefix=column)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df


In [47]:
onehot_encode(combined,columns=['addr_state'])

,loan_amnt,Risk_Score,dti,emp_length,policy_code,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,3600.0,677.0,5.91,10+ years,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,24700.0,717.0,16.06,10+ years,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,20000.0,697.0,10.78,10+ years,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10400.0,697.0,25.37,3 years,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,11950.0,692.0,10.20,4 years,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88940,25000.0,670.0,26.96,10+ years,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
88941,4000.0,676.0,64.48,10+ years,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
88942,5000.0,606.0,21.90,3 years,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
88943,2800.0,727.0,31.33,5 years,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
def ordinal_encode(df):
    df['emp_length'] = df['emp_length'].apply(lambda x: emp_ordering.index(x))

In [49]:

emp_ordering = [
    '< 1 year',
    '1 year',
    '2 years',
    '3 years',
    '4 years',
    '5 years',
    '6 years',
    '7 years',
    '8 years',
    '9 years',
    '10+ years'
]


In [50]:
ordinal_encode(combined)

In [51]:
combined

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,3600.0,677.0,5.91,PA,10,1
1,24700.0,717.0,16.06,SD,10,1
2,20000.0,697.0,10.78,IL,10,1
3,10400.0,697.0,25.37,PA,3,1
4,11950.0,692.0,10.20,GA,4,1
...,...,...,...,...,...,...
88940,25000.0,670.0,26.96,MA,10,0
88941,4000.0,676.0,64.48,PA,10,0
88942,5000.0,606.0,21.90,AZ,3,0
88943,2800.0,727.0,31.33,FL,5,0


In [52]:
label_encoder = LabelEncoder()

combined['policy_code'] = label_encoder.fit_transform(combined['policy_code'])

target_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}
target_mappings

{0: 0, 1: 1}

In [53]:
ordinal_encode(rejected)

<ipython-input-48-a5ba71b1f3a8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emp_length'] = df['emp_length'].apply(lambda x: emp_ordering.index(x))


In [54]:
rejected

,loan_amnt,Risk_Score,dti,addr_state,emp_length,policy_code
0,1000.0,693.0,10.00,NM,4,0
1,1000.0,703.0,10.00,MA,0,0
2,11000.0,715.0,10.00,MD,1,0
3,6000.0,698.0,38.64,MA,0,0
4,1500.0,509.0,9.43,MD,0,0
...,...,...,...,...,...,...
88940,25000.0,670.0,26.96,MA,10,0
88941,4000.0,676.0,64.48,PA,10,0
88942,5000.0,606.0,21.90,AZ,3,0
88943,2800.0,727.0,31.33,FL,5,0


In [55]:
label_encoder = LabelEncoder()

rejected['policy_code'] = label_encoder.fit_transform(rejected['policy_code'])

target_mappings2 = {index: label for index, label in enumerate(label_encoder.classes_)}
target_mappings2

<ipython-input-55-30b7466a0a95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejected['policy_code'] = label_encoder.fit_transform(rejected['policy_code'])


{0: 0}

In [56]:
def preprocess_inputs(df):
    df = df.copy()

#one hot encoding
    df = onehot_encode(
        df,
        columns=['addr_state']
    )
   
 # Split df into X and y
    y = df['policy_code']
    X = df.drop('policy_code', axis=1)
    
    return X, y

In [57]:
train_df = combined
test_df = rejected

In [58]:
X_train, y_train = preprocess_inputs(train_df)
X_test, y_test = preprocess_inputs(test_df)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [59]:
X_train

,loan_amnt,Risk_Score,dti,emp_length,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,-1.024204,0.301060,-0.020657,1.395187,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.199280,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.07472,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,4.814051,-0.065279,-0.125065,-0.046650,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
1,1.481483,0.592102,-0.020240,1.395187,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.199280,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.07472,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,21.436014,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
2,0.923344,0.446581,-0.020457,1.395187,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.199280,-0.070261,-0.027739,-0.018612,4.731660,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.07472,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.046650,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
3,-0.216684,0.446581,-0.019858,-0.431037,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.199280,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.07472,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,4.814051,-0.065279,-0.125065,-0.046650,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
4,-0.032617,0.410201,-0.020481,-0.170148,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,5.018068,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.07472,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.046650,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170381,1.517109,0.250128,-0.019793,1.395187,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.199280,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,6.463614,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0

In [60]:
y_train

0        1
1        1
2        1
3        1
4        1
        ..
88940    0
88941    0
88942    0
88943    0
88944    0
Name: policy_code, Length: 170386, dtype: int64

In [61]:
X_test

,loan_amnt,Risk_Score,dti,emp_length,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,-1.332962,0.417477,-0.020489,-0.170148,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.19928,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,13.383275,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.04665,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
1,-1.332962,0.490238,-0.020489,-1.213704,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.19928,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,6.463614,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.074720,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.04665,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
2,-0.145432,0.577550,-0.020489,-0.952815,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.19928,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,6.264369,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.074720,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.04665,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
3,-0.739197,0.453857,-0.019314,-1.213704,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.19928,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,6.463614,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.074720,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.04665,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
4,-1.273585,-0.921315,-0.020513,-1.213704,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.19928,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,-0.154712,6.264369,-0.049233,-0.167927,-0.130497,-0.139927,-0.074442,-0.054251,-0.133944,-0.040716,-0.056071,-0.074242,-0.193412,-0.074720,-0.108209,-0.292764,-0.195654,-0.098644,-0.105142,-0.207725,-0.065279,-0.125065,-0.04665,-0.100954,-0.300539,-0.084817,-0.178583,-0.044118,-0.145829,-0.124069,-0.06937,-0.043105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88940,1.517109,0.250128,-0.019793,1.395187,-0.047028,-0.120559,-0.093511,-0.153355,-0.383509,-0.146541,-0.11973,-0.050709,-0.056543,-0.283675,-0.19928,-0.070261,-0.027739,-0.018612,-0.211342,-0.104541,-0.091053,-0.108348,-0.109945,6.463614,-0.159633,-0.049233,-0.167927,-0.130497,-0.139927,-0.

In [62]:
y_test

0        0
1        0
2        0
3        0
4        0
        ..
88940    0
88941    0
88942    0
88943    0
88944    0
Name: policy_code, Length: 88945, dtype: int64

In [63]:
models = {
#"   K-Nearest Neighbors": KNeighborsClassifier(),
  "   Logistic Regression": LogisticRegression(),
  #  "Support Vector Machine": SVC(),
    "         Random Forest": RandomForestClassifier(),
   
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

   Logistic Regression trained.
         Random Forest trained.


In [64]:
for name, model in models.items():
  print(name + " Accuracy: {:.2f}%".format(model.score(X_test, y_test) * 100))

   Logistic Regression Accuracy: 82.92%
         Random Forest Accuracy: 100.00%


In [65]:
models2 = {
"   K-Nearest Neighbors": KNeighborsClassifier(),
 # "   Logistic Regression": LogisticRegression(),
  #  "Support Vector Machine": SVC(),
   # "         Decision Tree": DecisionTreeClassifier(),
   
}

for name, model in models2.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

   K-Nearest Neighbors trained.


In [66]:
for name, model in models2.items():
  print(name + " Accuracy: {:.2f}%".format(model.score(X_test, y_test) * 100))

   K-Nearest Neighbors Accuracy: 88.43%
